<a href="https://colab.research.google.com/github/abdur75648/AI-Camera/blob/main/colab_notebooks/DocRes_OCR_Data_Enhancement_Sarvam_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!git clone https://github.com/ZZZHANG-jx/DocRes.git
%cd DocRes

Cloning into 'DocRes'...
remote: Enumerating objects: 300, done.
remote: Counting objects: 100% (40/40), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 300 (delta 27), reused 19 (delta 19), pack-reused 260 (from 1)
Receiving objects: 100% (300/300), 223.98 MiB | 19.80 MiB/s, done.
Resolving deltas: 100% (59/59), done.
Updating files: 100% (143/143), done.
/content/DocRes


In [ ]:
!mkdir checkpoints
!mkdir data/MBD/checkpoint/
!cd data/MBD/checkpoint/ && gdown 1gDnjHZdMOzrNr_mx0qm2zCYdGxGykY0E
!cd checkpoints && gdown 17zZfc7nj-5BrrnD1Ax22zdDG73NTbkir

Downloading...
From (original): https://drive.google.com/uc?id=1gDnjHZdMOzrNr_mx0qm2zCYdGxGykY0E
From (redirected): https://drive.google.com/uc?id=1gDnjHZdMOzrNr_mx0qm2zCYdGxGykY0E&confirm=t&uuid=66bbeb7f-646f-4385-9ddf-c048f3133972
To: /content/DocRes/data/MBD/checkpoint/mbd.pkl
100% 713M/713M [00:04<00:00, 149MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=17zZfc7nj-5BrrnD1Ax22zdDG73NTbkir
From (redirected): https://drive.google.com/uc?id=17zZfc7nj-5BrrnD1Ax22zdDG73NTbkir&confirm=t&uuid=6895a6a4-b329-407f-ad48-201d49a3b837
To: /content/DocRes/checkpoints/docres.pkl
100% 183M/183M [00:01<00:00, 140MB/s]


In [ ]:
# Modifying the inference.py to fix the "DEICE" variable error
with open('inference.py', 'r') as f:
    content = f.read()
functions_to_modify = [
    'model_init',
    'dewarping',
    'appearance',
    'deshadowing',
    'deblurring',
    'binarization',
    'inference_one_im'
]

# Add 'DEVICE' as argument to each function definition
for func in functions_to_modify:
    content = content.replace(f'def {func}(', f'def {func}(')
    if func != 'model_init':
        content = content.replace(f'def {func}(model,im_path', f'def {func}(model,im_path,DEVICE')
    else:
        content = content.replace('def model_init(args):', 'def model_init(args, DEVICE):')
content = content.replace('inference_one_im(model,im_path,task):', 'inference_one_im(model,im_path,task,DEVICE):')
tasks = ['dewarping', 'deshadowing', 'appearance', 'deblurring', 'binarization']
for task in tasks:
    content = content.replace(f'{task}(model,im_path)', f'{task}(model,im_path,DEVICE)')

with open('inference.py', 'w') as f:
    f.write(content)

In [ ]:
# Re-start Runtime for Updated inference.py
import os
os.kill(os.getpid(), 9)

In [ ]:
%cd DocRes

/content/DocRes


In [ ]:
import os
import cv2
import torch
import tempfile
from argparse import Namespace
from inference import model_init, inference_one_im

drive_base_path = '/content/drive/MyDrive/Sarvam-AI-VLM-Assignment/dataset/'
input_imgs_dir = os.path.join(drive_base_path, 'raw_images')
output_imgs_dir = os.path.join(drive_base_path, 'enhanced_images')
assert os.path.exists(input_imgs_dir), f"{input_imgs_dir} directory doesn't exist"
os.makedirs(output_imgs_dir, exist_ok=True)

# --- Create a Namespace object to hold our arguments ---
args = Namespace(
    # Path to the pre-trained model checkpoint
    model_path='./checkpoints/docres.pkl',

    # Folder where your input images are located
    im_path=input_imgs_dir,

    # Folder where the final outputs will be saved
    out_folder=output_imgs_dir,

    # Not Used
    task=None,
    save_dtsprompt=0
)

# --- Setup the device (GPU if available, otherwise CPU) ---
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {DEVICE}")
print(f"Input folder set to: {args.im_path}")
print(f"Output folder set to: {args.out_folder}")

Using device: cuda
Input folder set to: /content/drive/MyDrive/Sarvam-AI-VLM-Assignment/dataset/raw_images
Output folder set to: /content/drive/MyDrive/Sarvam-AI-VLM-Assignment/dataset/enhanced_images


In [ ]:
print("Initializing the DocRes model...")
model = model_init(args, DEVICE)
print("Model loaded successfully.")

Initializing the DocRes model...
Model loaded successfully.


In [ ]:
def process_image_sequentially(model, input_image_path, output_folder, DEVICE):
    """
    We'll run appearance -> deblurring -> binarization tasks sequentially.
    Args:
        model: The loaded DocRes model.
        input_image_path (str): Path to the single input image.
        output_folder (str): Folder to save the final output.

    NOTE - Resizing the images to not have more than 1024 width to avoid CUDA out of memory error
    """
    if not os.path.exists(input_image_path):
        print(f"Error: Input file not found at {input_image_path}")
        return

    print(f"Processing {os.path.basename(input_image_path)}...")


    with tempfile.TemporaryDirectory() as temp_dir:
        img = cv2.imread(input_image_path)
        h, w = img.shape[:2]
        if w > 1024:
            # print(f"Image is large ({w}x{h}). Resizing to 1024px width to avoid GPU Out of Memory Error!")
            new_w = 1024
            new_h = int(h * (new_w / w))
            img = cv2.resize(img, (new_w, new_h), interpolation=cv2.INTER_AREA)
        resized_input_path = os.path.join(temp_dir, 'step0_resized.png')
        cv2.imwrite(resized_input_path, img)
        del img

        temp_path_step1 = os.path.join(temp_dir, 'temp_step1.png')
        temp_path_step2 = os.path.join(temp_dir, 'temp_step2.png')
        im_name = os.path.basename(input_image_path)
        final_save_path = os.path.join(output_folder, f"{os.path.splitext(im_name)[0]}.png")


        # --- Step 1: Appearance Correction ---
        print("Step 1: Applying Appearance Correction...")
        _, _, _, restored_step1 = inference_one_im(model, resized_input_path, DEVICE, 'appearance')
        cv2.imwrite(temp_path_step1, restored_step1)
        del restored_step1
        torch.cuda.empty_cache()

        # --- Step 2: Deblurring ---
        print("Step 2: Applying Deblurring...")
        _, _, _, restored_step2 = inference_one_im(model, temp_path_step1, DEVICE, 'deblurring')
        cv2.imwrite(temp_path_step2, restored_step2)
        del restored_step2
        torch.cuda.empty_cache()

        # --- Step 3: Binarization ---
        print("Step 3: Applying Binarization...")
        _, _, _, final_restored = inference_one_im(model, temp_path_step2, DEVICE, 'binarization')

        # --- Save the final result ---
        cv2.imwrite(final_save_path, final_restored)
        print(f"✅ Final image saved to: {final_save_path}")
        del final_restored
        torch.cuda.empty_cache()

In [ ]:
# Run Inference on All Images -> For simplicity running one-by-one
image_files = sorted([f for f in os.listdir(args.im_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))])

if not image_files:
    print(f"No images found in {args.im_path}. Please check the path and upload images.")
else:
    print(f"Found {len(image_files)} images to process.")

for image_name in image_files:
    full_input_path = os.path.join(args.im_path, image_name)

    # Run the full processing chain on the current image
    process_image_sequentially(
        model=model,
        input_image_path=full_input_path,
        output_folder=args.out_folder,
        DEVICE=DEVICE
    )
    print("-" * 30)

print("All images processed successfully!")

Found 100 images to process.
Processing Swarupsannidhan_0001.png...
Step 1: Applying Appearance Correction...
Step 2: Applying Deblurring...
Step 3: Applying Binarization...
✅ Final image saved to: /content/drive/MyDrive/Sarvam-AI-VLM-Assignment/dataset/enhanced_images/Swarupsannidhan_0001.png
------------------------------
Processing Swarupsannidhan_0002.png...
Step 1: Applying Appearance Correction...
Step 2: Applying Deblurring...
Step 3: Applying Binarization...
✅ Final image saved to: /content/drive/MyDrive/Sarvam-AI-VLM-Assignment/dataset/enhanced_images/Swarupsannidhan_0002.png
------------------------------
Processing Swarupsannidhan_0003.png...
Step 1: Applying Appearance Correction...
Step 2: Applying Deblurring...
Step 3: Applying Binarization...
✅ Final image saved to: /content/drive/MyDrive/Sarvam-AI-VLM-Assignment/dataset/enhanced_images/Swarupsannidhan_0003.png
------------------------------
Processing Swarupsannidhan_0004.png...
Step 1: Applying Appearance Correction...